# Adicionando funções externas utilizando LangChain

LangChain é um framework para criação de aplicações de IA e naturalmente ele possui ferramentas para facilitar a criação de funções externas para serem passadas a api da OpenAI. Para a utilização do LangChain, será necessário entendermos brevemente uma outra biblioteca de Python chamada pydantic, uma bilioteca para validação de dados que facilita a construção de estruturas de dados mais robustas.

## Introduzindo pydantic

### Como criamos uma estrutura nova sem pydantic

Sem utilizar pydantic, podemos criar uma estrutura de dados utilizando funções da seguinte forma:

In [1]:
class Pessoa:
    def __init__(self, nome: str, idade: int, peso: float) -> None:
        self.nome = nome
        self.idade = idade
        self.peso = peso

Neste caso, criamos uma função que representa uma pessoa e tem os seguintes atributos: nome, idade e peso.

In [2]:
adriano = Pessoa('Adriano', 32, 68)
adriano

In [3]:
adriano.idade

32

Podemos setar os atributos com qualquer tipo de dado, pois o Python por padrão não faz checagem de tipos.

In [4]:
adriano = Pessoa('Adriano', 32, 'ashdbadgvuya')
adriano

In [5]:
adriano.peso

'ashdbadgvuya'

### Como criamos uma estrutura nova usando pydantic

A sintaxe de pydantic acaba sendo bem mais simples para a criação de classes de dados, ao compararmos com a criação de classes comuns de Python. Nela, temos que cuidar com a definição do tipo de cada atributo, pois eles serão utilizados para validar se os dados fornecidos estão corretos.

In [6]:
from pydantic import BaseModel

class pydPessoa(BaseModel):
    nome: str
    idade: int
    peso: float

In [9]:
adriano = pydPessoa(nome='Adriano', idade=32, peso=68)
adriano

pydPessoa(nome='Adriano', idade=32, peso=68.0)

In [10]:
adriano.nome

'Adriano'

O interessante é vermos que pydantic fornece uma validação automática de dados. Isso garante uma integridade muito maior em aplicações mais complexas.

In [12]:
adriano = pydPessoa(nome='Adriano', idade=32, peso='asuhdauishg')


ValidationError: 1 validation error for pydPessoa
peso
  Input should be a valid number, unable to parse string as a number [type=float_parsing, input_value='asuhdauishg', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/float_parsing

In [13]:
adriano = pydPessoa(nome='Adriano', idade=32, peso='68')
adriano.peso

68.0

Podemos fazer um nesting de classes de pydantic, onde uma classe de dados recebe como input outra classe de pydantic.

In [14]:
from typing import List

class pydAsimoTeam(BaseModel):
    funcionarios: List[pydPessoa]

pydAsimoTeam(funcionarios=[pydPessoa(nome='Adriano', idade=32, peso=68)])

pydAsimoTeam(funcionarios=[pydPessoa(nome='Adriano', idade=32, peso=68.0)])

E a validação continua fucnionando mesmo com esta estrutura de nesting.

In [12]:
pydAsimoTeam(funcionarios=[Pessoa(nome='Adriano', idade=32, peso=68)])

ValidationError: 1 validation error for pydAsimoTeam
funcionarios.0
  Input should be a valid dictionary or instance of pydPessoa [type=model_type, input_value=<__main__.Pessoa object at 0x7f126b89f190>, input_type=Pessoa]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type

## Utilizando pydantic para criação de tools da OpenAI

Video 10 minutos 

In [15]:
import json

def obter_temperatura_atual(local, unidade="celsius"):
    if "são paulo" in local.lower():
        return json.dumps(
            {"local": "São Paulo", "temperatura": "32", "unidade": unidade}
            )
    elif "porto alegre" in local.lower():
        return json.dumps(
            {"local": "Porto Alegre", "temperatura": "25", "unidade": unidade}
            )
    else:
        return json.dumps(
            {"local": local, "temperatura": "unknown"}
            )
    
tools = [
    {
        "type": "function",
        "function": {
            "name": "obter_temperatura_atual",
            "description": "Obtém a temperatura atual em uma dada cidade",
            "parameters": {
                "type": "object",
                "properties": {
                    "local": {
                        "type": "string",
                        "description": "O nome da cidade. Ex: São Paulo",
                    },
                    "unidade": {
                        "type": "string", 
                        "enum": ["celsius", "fahrenheit"]
                    },
                },
                "required": ["local"],
            },
        },
    }
    ]


In [16]:
from pydantic import BaseModel, Field #Importação atualizada
from typing import Optional
from enum import Enum

class UnidadeEnum(str, Enum):
    celsius = 'celsius'
    fahrenheit = 'fahrenheit'

class ObterTemperaturaAtual(BaseModel):
    """Obtém a temperatura atual de uma determinada localidade"""
    local: str = Field(description='O nome da cidade', examples=['São Paulo', 'Porto Alegre'])
    unidade: Optional[UnidadeEnum]

In [24]:
from langchain_core.utils.function_calling import convert_to_openai_function

tool_temperatura = convert_to_openai_function(ObterTemperaturaAtual)
tool_temperatura

{'name': 'ObterTemperaturaAtual',
 'description': 'Obtém a temperatura atual de uma determinada localidade',
 'parameters': {'properties': {'local': {'description': 'O nome da cidade',
    'examples': ['São Paulo', 'Porto Alegre'],
    'type': 'string'},
   'unidade': {'anyOf': [{'enum': ['celsius', 'fahrenheit'],
      'title': 'UnidadeEnum',
      'type': 'string'},
     {'type': 'null'}]}},
  'required': ['local', 'unidade'],
  'type': 'object'}}

## Adicionando função externa utilizando LangChain 

Agora que já sabemos criar funções que os modelos de llm entendam, podemos passar essas funções para os modelos de linguagem através da biblioteca langchain. Para isso temos duas formas, podemos utilizar o parâmetro functions ao chamar o método invoke dos chat_models:

In [25]:


from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template('Crie uma frase sobre o seguinte: {assunto}')

chain = prompt | model

response = chain.invoke('Qual é a temperatura de Porto Alegre', functions=[tool_temperatura])
response.content

'Para saber a temperatura exata em Porto Alegre agora, consulte um serviço de meteorologia online.'

Ou podemos dar um bind e criar um novo componente de chat_model que terá acesso a função sempre que for chamado o invoke. Nestes dois casos, o modelo se comportará com o parâmetro "auto" de chamamento de função, ou seja, ele chamará a função quando necessitar, caso contrário se comportará como um modelo de linguagem normal.

In [27]:
# chat = ChatOpenAI()
# chat_com_func = chat.bind(functions=[tool_temperatura])
# resposta = chat_com_func.invoke('Qual é a temperatura de Porto Alegre')
# resposta
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template('Crie uma frase sobre o seguinte: {assunto}')

chain = prompt | model

chat_com_func = chain.bind(functions=[tool_temperatura])

response = chain.invoke('Qual é a temperatura de Porto Alegre')
response


AIMessage(content='A temperatura em Porto Alegre hoje é de [inserir temperatura] e [inserir condição climática].', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--1f385240-1d4b-4e1f-afd3-9ae834232597-0', usage_metadata={'input_tokens': 15, 'output_tokens': 22, 'total_tokens': 37, 'input_token_details': {'cache_read': 0}})

Podemos obrigar o modelo a sempre chamar uma função da seguinte forma:

In [28]:
resposta = chain.invoke(
    'Qual é a temperatura de Porto Alegre', 
    functions=[tool_temperatura],
    function_call={'name': 'ObterTemperaturaAtual'}
    )
resposta

AIMessage(content='A temperatura em Porto Alegre hoje é um convite para aproveitar a cidade, seja com um chimarrão no parque ou explorando seus recantos culturais.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--74f79faf-f94e-4a85-a76f-bb89f6c4bed4-0', usage_metadata={'input_tokens': 15, 'output_tokens': 32, 'total_tokens': 47, 'input_token_details': {'cache_read': 0}})

In [29]:
resposta = chain.invoke(
    'Olá', 
    functions=[tool_temperatura],
    function_call={'name': 'ObterTemperaturaAtual'}
    )
resposta

AIMessage(content='Aqui estão algumas opções de frases sobre "Olá", com diferentes tons:\n\n*   **Simples e direta:** "Olá" é a saudação mais básica e universal.\n*   **Enfatizando a conexão:** Um simples "Olá" pode ser o início de uma grande conversa.\n*   **Com um toque de mistério:** Um sussurro de "Olá" ecoou no silêncio da noite.\n*   **Sobre a importância da saudação:** Nunca subestime o poder de um "Olá" sincero.\n*   **Com humor:** "Olá", disse ele, como se estivesse descobrindo a palavra naquele instante.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--f0a1b2b6-fa4b-4952-8be8-39658017be0e-0', usage_metadata={'input_tokens': 10, 'output_tokens': 136, 'total_tokens': 146, 'input_token_details': {'cache_read': 0}})

### Adicionando a uma chain

Podemos adicionar agora este modelo com funções a um prompt e criar uma chain.

In [30]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ('system', 'Você é um assistente amigável chamado Isaac'),
    ('user', '{input}')
])

chain = prompt | chain.bind(functions=[tool_temperatura])

In [31]:
chain.invoke({'input': 'Olá'})

AIMessage(content='"Olá!", respondeu Isaac, o assistente amigável, pronto para ajudar com um sorriso digital.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--63cd7ab3-e74b-42b4-936a-8af494a6319e-0', usage_metadata={'input_tokens': 49, 'output_tokens': 20, 'total_tokens': 69, 'input_token_details': {'cache_read': 0}})

In [32]:
chain.invoke({'input': 'Qual a temperatura em Floripa?'})

AIMessage(content='"Isaac, o assistente amigável, recebeu a pergunta sobre a temperatura em Floripa."', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--d7a7f592-d782-4e5d-a44b-5ccfcd78068c-0', usage_metadata={'input_tokens': 54, 'output_tokens': 19, 'total_tokens': 73, 'input_token_details': {'cache_read': 0}})